## Multioutput model

Now, we need a model to detect the type of hate

In [1]:
%load_ext autoreload
%autoreload 2
import json

with open("../data/train.json") as f:
    train_articles = json.load(f)

with open("../data/test.json") as f:
    test_articles = json.load(f)

Let's take just the comments that are HATEFUL

In [2]:
from sklearn.model_selection import train_test_split
from datasets import Dataset, Value, ClassLabel, Features
import pandas as pd

def serialize(article, comment):
    ret = comment.copy()
    ret["context"] = article["title"]
    return ret
    

train_comments = [serialize(article, comment) for article in train_articles for comment in article["comments"] if comment["is_hateful"]]
test_comments = [serialize(article, comment) for article in test_articles for comment in article["comments"] if comment["is_hateful"]]


train_df = pd.DataFrame(train_comments)
test_df = pd.DataFrame(test_comments)

train_df, dev_df = train_test_split(train_df, test_size=0.2, random_state=20212021)

print(f"We have {len(train_df)} hateful comments in train")
print(f"We have {len(dev_df)} hateful comments in dev")
print(f"We have {len(test_df)} hateful comments in test")

We have 5140 hateful comments in train
We have 1285 hateful comments in dev
We have 1676 hateful comments in test


In [3]:


train_df[categories].mean() - test_df[categories].mean()

calls         0.041225
WOMEN        -0.044536
LGBTI        -0.022262
RACISM        0.038171
CLASS        -0.018981
POLITICS     -0.024523
DISABLED     -0.011560
APPEARANCE   -0.010102
CRIMINAL      0.077152
dtype: float64

It is slightly unbalanced!

In [4]:
from hatedetection.preprocessing import preprocess_tweet

for df in [train_df, dev_df, test_df]:
    df["text"] = df["text"].apply(preprocess_tweet)
    df["context"] = df["context"].apply(preprocess_tweet)


## Clasificación

Usamos nuestro modelo `hatedetection.BertForSequenceMultiClassification`. Es una leve modificación del clasificador de `transformers`

In [5]:
import torch

from transformers import AutoTokenizer
from hatedetection import BertForSequenceMultiClassification

model_name = 'dccuchile/bert-base-spanish-wwm-cased'

device = "cuda" if torch.cuda.is_available() else "cpu"

id2label = {0: 'Not hateful', 1: 'Hateful'}
label2id = {v:k for k,v in id2label.items()}

model = BertForSequenceMultiClassification.from_pretrained(model_name, return_dict=True, num_labels=len(categories))

model.config.id2label = id2label
model.config.label2id = label2id

model = model.to(device)
model.train();



tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.model_max_length = 256

Some weights of the model checkpoint at dccuchile/bert-base-spanish-wwm-cased were not used when initializing BertForSequenceMultiClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceMultiClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceMultiClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceMultiClassification were not initialized from the model checkpoint at dccuchile/bert-base

In [6]:
from datasets import Dataset, Value, ClassLabel, Features

#examples = pd.concat([train_df, dev_df])

features = Features({
    'text': Value('string'),
    'context': Value('string'),
})

for cat in categories:
    features[cat] = ClassLabel(num_classes=2, names=["NO", "YES"])

columns = ["context", "text"] + categories

train_dataset = Dataset.from_pandas(train_df[columns], features=features)
dev_dataset = Dataset.from_pandas(dev_df[columns], features=features)
test_dataset = Dataset.from_pandas(test_df[columns], features=features)


In [7]:
def tokenize(batch):
    return tokenizer(batch['context'], batch['text'], padding='max_length', truncation=True)

batch_size = 32
eval_batch_size = 16

train_dataset = train_dataset.map(tokenize, batched=True, batch_size=batch_size)
dev_dataset = dev_dataset.map(tokenize, batched=True, batch_size=eval_batch_size)
test_dataset = test_dataset.map(tokenize, batched=True, batch_size=eval_batch_size)



In [8]:
def format_dataset(dataset):
    def sarasa(examples):
        return {'labels': torch.Tensor([examples[cat] for cat in categories])}
    dataset = dataset.map(sarasa)
    dataset.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
    return dataset

train_dataset = format_dataset(train_dataset)
dev_dataset = format_dataset(dev_dataset)
test_dataset = format_dataset(test_dataset)

Esta API de mierda vive cambiando todo el tiempo

In [9]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

def compute_metrics(pred):
    """
    Compute metrics for Trainer
    """    
    labels = pred.label_ids
    preds = torch.sigmoid(torch.Tensor(pred.predictions)).round()

    ret = {
    }
    """
    Calculo F1 por cada posición. Asumo que cada categoría está alineada correctamente en la i-ésima posición
    """
    f1s = []
    for i, cat in enumerate(categories):
        cat_labels, cat_preds = labels[:, i], preds[:, i]
        precision, recall, f1, _ = precision_recall_fscore_support(cat_labels, cat_preds, average='macro')
        
        f1s.append(f1)
        
        ret[cat+" F1"] = f1
        
    ret["Mean F1"] = torch.Tensor(f1s).mean()
    return ret

In [10]:
from transformers import Trainer, TrainingArguments
epochs = 5

total_steps = (epochs * len(train_dataset)) // batch_size
warmup_steps = total_steps // 10

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=eval_batch_size,
    warmup_steps=warmup_steps,
    evaluation_strategy="epoch",
    do_eval=False,
    weight_decay=0.01,
    logging_dir='./logs',
)

results = []

trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=dev_dataset,
)

trainer.train()


Epoch,Training Loss,Validation Loss,Calls f1,Women f1,Lgbti f1,Racism f1,Class f1,Politics f1,Disabled f1,Appearance f1,Criminal f1,Mean f1,Runtime,Samples Per Second
1,No log,0.181724,0.864354,0.801104,0.906677,0.949633,0.762255,0.832120,0.663099,0.866771,0.960744,0.845195,10.892400,117.972000
2,No log,0.131913,0.864543,0.875970,0.931996,0.952856,0.841412,0.872669,0.839191,0.928037,0.972235,0.897657,10.765900,119.359000
3,No log,0.120396,0.878464,0.894050,0.934324,0.956837,0.875821,0.875178,0.886712,0.931112,0.975076,0.911953,10.838100,118.563000
4,0.191600,0.113226,0.886335,0.908481,0.934324,0.961118,0.876004,0.893410,0.892455,0.935192,0.976230,0.918172,10.875800,118.152000
5,0.191600,0.113917,0.890694,0.900315,0.932587,0.961289,0.892798,0.897183,0.877159,0.937336,0.971790,0.917906,10.825900,118.697000


TrainOutput(global_step=805, training_loss=0.138434038102997, metrics={'train_runtime': 721.588, 'train_samples_per_second': 1.116, 'total_flos': 4336658666035200.0, 'epoch': 5.0, 'init_mem_cpu_alloc_delta': 54563, 'init_mem_gpu_alloc_delta': 0, 'init_mem_cpu_peaked_delta': 18258, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 611560, 'train_mem_gpu_alloc_delta': 1327355392, 'train_mem_cpu_peaked_delta': 191435828, 'train_mem_gpu_peaked_delta': 9070782464})

In [38]:
def predict(sentence, context=None):
    context = preprocess_tweet(context)
    sentence = preprocess_tweet(sentence)
    idx = torch.LongTensor(tokenizer.encode(context, sentence)).view(1, -1).to(device)
    output = model(idx)

    output = output.logits.detach().cpu().numpy()[0]

    ret = [cat for cat, out in list(zip(categories, output > 0)) if out]

    return ret

In [39]:
predict("Negro chorro de mierda", context="Motín de presos en Devoto")

['RACISM', 'CRIMINAL']

In [41]:
predict("Quiere bijaaa", context="Florencia Peña dio una entrevista a Infobae")

['WOMEN']

In [42]:
predict("Le falta verga", context="Cristiano Ronaldo metió su gol número 1000")

['LGBTI']

In [43]:
predict("Putaldo", context="Cristiano Ronaldo metió su gol número 1000")

['LGBTI']

In [44]:
predict("Negro sucio", context="Cristiano Ronaldo metió su gol número 1000")

['RACISM']

In [50]:
predict("Romina tiene cuerpo de camionero", context="Romina C nos cuenta sus vacaciones en Cancún")

['WOMEN']

In [51]:
predict("Sos macho Romina", context="Romina C nos cuenta sus vacaciones en Cancún")

['WOMEN', 'LGBTI']

In [52]:
predict("Callate varón", context="Romina C nos cuenta sus vacaciones en Cancún")

['WOMEN', 'LGBTI']

In [59]:
predict("Romina viene con sorpresa entre las dos gambas", context="Romina C nos cuenta sus vacaciones en Cancún")

['WOMEN']

In [61]:
predict("Viene con paquete entre las dos gambas", context="Romina C nos cuenta sus vacaciones en Cancún")

['WOMEN']

In [63]:
predict("es el defensor de Cambáceres", context="Romina C nos cuenta sus vacaciones en Cancún")

['WOMEN']

Falla

In [56]:
predict("A esta trola hay que agarrarla de a varios para que sepa lo que es bueno", context="Romina C nos cuenta sus vacaciones en Cancún")

['WOMEN']

Acá no encuentra el llamado a la acción...ahora, si lo cambiamos ligeramente...

In [58]:
predict("A esta negra trola hay que agarrarla de a varios para que sepa lo que es bueno", context="Romina C nos cuenta sus vacaciones en Cancún")

['WOMEN', 'RACISM']

In [20]:
predict("Hay que encerrar a todos estos homosexuales de mierda")

['LGBTI', 'CRIMINAL']

In [21]:
predict("Qué asco de homosexuales")

['LGBTI']

In [22]:
predict("Qué asco de homosexuales, mátenlos")

['LGBTI']

Parece que correlaciona todo llamado a la acción a los criminales

In [23]:
predict("Negros chorros de mierda, cuarenta ladrillos y arreglamos esto")

['RACISM']

In [24]:
predict("negras al paredón!")

['RACISM']

In [25]:
predict("No queremos rojitos acá")

['RACISM', 'POLITICS']

In [26]:
predict("No les gusta laburar eh")

['CLASS']

In [27]:
predict("Agarrá la pala!")

[]

In [28]:
predict("Agarrá la pala, vago!")

['CLASS', 'CRIMINAL']

In [29]:
predict("Menos ganas de laborar eh")

[]

In [30]:
predict("Viven del Estado, esta gente sinceramente no tiene remedio")

['CLASS']

In [31]:
predict("Tenés dos pelotas entre las piernas amigazo")

['LGBTI']

In [34]:
tokenizer.decode(tokenizer.encode("🤢"))

'[CLS] [UNK] [SEP]'